<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BDII_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025年度後期 人工知能II 試験問題
以下の問いに答えよ。解答は本ファイルに直接記入し、ダウンロードした ipynb ファイルを提出すること。

参考：[Google Colab での数式の書き方](https://hwb.ecc.u-tokyo.ac.jp/hwb2023/applications/latex/5min/)

## 問１ ニューラルネットワークに関する数式

単一の実数値入力と単一の実数値出力を持つReLUネットワークによって表される関数を考える。$l$ を層の数とすると、例えば $l=2$ のとき、ネットワークは次のように記述される。

$$
f(x; W_1, W_2, b_1, b_2) = W_2 \text{ReLU}(W_1 x + b_1) + b_2
$$

この時、以下の問いに答えよ。

（１）
幅 $k$（すなわち $k$ 個の隠れニューロン）を持ち、重み行列 $W_1, W_2$ およびバイアスベクトル $b_1, b_2$ を備えた2層ReLUネットワークを考える。$W_1$ は $\mathbb{R}^{k \times 1}$ の行列であるとき、$W_2, b_1, b_2$ の形状を書きなさい。

（解答欄）


（２）
$l=3$ のReLUネットワークの式を書きなさい。

（解答欄）


（３）
各層の幅が $k$ である $l$ 層のReLUネットワークを考える。入力に対して、出力は最大でいくつの不連続点を持つ可能性があるか？


（解答欄）


（４）2次元入力 $x \in \mathbb{R}^2$ を持ち、以下の条件を満たす2層・幅2のReLUネットワークを作成し、具体的な $W_1, W_2, b_1, b_2$ を用いて $f$ を表せ 。

$$条件：f(x; W_1, W_2, b_1, b_2) > 0 \iff x_1 > 0 \text{ または } x_2 > 0$$

（解答欄）


## 問２ ニューラルネットワークに関するプログラミング

In [ ]:
# 準備
import torch
assert torch.cuda.is_available(), "Should use GPU-enabled colab"
device = torch.device('cuda:0')  # we will train with CUDA!

以下に示す Module クラスは、PyTorch の nn.Module と似た API を持つが、autograd（自動微分）を使用する代わりに、バックプロパゲーションの操作（module.backward）を手動で実装する点が異なる。

In [ ]:
from typing import *
import abc


# This is our Module API
class Module(abc.ABC):
    device: Optional[torch.device]  # Parameters should live on this device!
    inputs: Tuple[torch.Tensor, ...]

    def __init__(self, device=None):
        self.device = device

    @abc.abstractmethod
    def parameters(self) -> Iterator[torch.Tensor]:
        r'''
        Returns an iterator over the *parameters* of this module.

        Subclass needs to implement this.
        '''

    @abc.abstractmethod
    def forward(self, *inputs: torch.Tensor) -> torch.Tensor:
        r'''
        Returns the output of applying this module on tensors `inputs`, each of
        which is a *batched* tensor.

        In most cases, the module takes a single input tensor (e.g., linear
        layer and ReLU layers). However, multiple inputs are useful when the
        module computes a loss between a prediction and groundtruth target.

        Subclass needs to implement this.
        '''

    def __call__(self, *inputs: torch.Tensor) -> torch.Tensor:
        r'''
        Simply calls forward, and stores inputs at `self.inputs`, which may be
        useful for computing gradients in `backward`.
        '''
        self.inputs = inputs
        return self.forward(*inputs)

    @abc.abstractmethod
    def backward(self, dLdout: torch.Tensor) -> torch.Tensor:
        r'''
        This is our manual backprop.

        Given, `dLdOut` as $dL / d output$, for some loss `L`, we compute
        1. For each parameter `p` of this module, compute $d L /d p$, stored at `p.grad`.
        2. $dL / d self.inputs[0]$, to be passed to the previous layer. Only
           needs to compute derivative of the first input.

        Note that $dL / d *$ should always be a tensor of same shape as *. E.g.,
        $d L /d p$ (i.e., `p.grad`) should always be of the same shape as `p`.

        Subclass needs to implement this.
        '''

    def zero_grad(self):
        r'''
        Clear any previous computed gradients.
        '''
        for p in self.parameters():
            p.grad = None

ニューラルネットワークを構成するために、以下の3種類のモジュールを作成する。

+ 線形層（全結合層）
+ ReLU活性化関数
+ クロスエントロピー損失

**問題：以下の各モジュールクラスにおいて、未完成の forward および backward の定義を完成させよ。各実装は5行以内のコードで行うこと。該当箇所には FIXME と記されている。**

In [ ]:
class Linear(Module):
    def __init__(self, in_features: int, out_features: int, device=None):
        super().__init__(device)
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.randn(out_features, in_features, device=device) / in_features  # weight matrix
        self.bias = torch.zeros(out_features, device=device)  # bias vector

    def parameters(self) -> Iterator[torch.Tensor]:
        return [self.weight, self.bias]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x           shape: [b, in_features]
        # self.weight shape: [out_features, in_features]
        # self.bias   shape: [out_features]
        #
        # output should have shape: [b, out_features]
        #
        # FIXME
        pass

    def backward(self, dLdout: torch.Tensor) -> torch.Tensor:
        # self.inputs[0] shape: [b, in_features]
        # dLdout         shape: [b, out_features]
        # self.weight    shape: [out_features, in_features]
        # self.bias      shape: [out_featurs]
        #
        # FIXME
        # Note that you should *not* modify `dLdout` or `self.inputs` inplace.
        pass

    def __repr__(self) -> str:
        return f"Linear(in={self.in_features}, out={self.out_features})"


class ReLU(Module):
    def parameters(self) -> Iterator[torch.Tensor]:
        return []

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x.clamp(min=0)

    def backward(self, dLdout: torch.Tensor) -> torch.Tensor:
        # self.inputs[0]  shape: [b, d]
        # dLdout          shape: [b, d]
        #
        # FIXME
        # Note that you should *not* modify `dLdout` or `self.inputs` inplace.
        pass

    def __repr__(self) -> str:
        return "ReLU()"


class CrossEntropyLoss(Module):
    def parameters(self) -> Iterator[torch.Tensor]:
        return []

    def forward(self, logits: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        # logits    shape: [b, num_classes]
        # target    shape: [b], containing *integers* in [0, 1, ..., num_classes - 1]
        #
        # For each logits in the batch
        #   p = softmax(logits)
        #   loss = -log(p[target])
        # Total loss is averaged across the entire batch.
        b = logits.shape[0]
        return -logits.softmax(dim=-1).log()[torch.arange(b, device=self.device), target].mean()  # scalar, shape: []

    def backward(self, dLdout: torch.Tensor) -> torch.Tensor:
        logits, target = self.inputs
        # logits    shape: [b, num_classes]
        # target    shape: [b], containing *integers* in [0, 1, ..., num_classes - 1]
        # dLdout    shape: []
        #
        # FIXME
        # Note that you should *not* modify `dLdout` or `self.inputs` inplace.
        # Compute dL / d logits
        pass

    def __repr__(self) -> str:
        return "CrossEntropyLoss()"

### 答え合わせ

実際にネットワークを作成して、上記の実装がうまくいっているかどうかを確認する。

In [ ]:
class Network(Module):
    layers: List[Module]

    def __init__(self, input_dim, device=None):
        super().__init__(device)
        self.layers = [
            Linear(input_dim, 8192, device=device),
            ReLU(device=device),
            Linear(8192, 8192, device=device),
            ReLU(device=device),
            Linear(8192, 10, device=device),
        ]

    def parameters(self):
        r'''
        Parameters are from layers.
        '''
        for layer in self.layers:
            yield from layer.parameters()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        r'''
        Sequentially activate all layers.
        '''
        y = x
        for layer in self.layers:
            y = layer(y)
        return y

    def backward(self, dLdout: torch.Tensor) -> torch.Tensor:
        r'''
        Backpropagation: backward through each layer in reverse order.
        '''
        for layer in self.layers[::-1]:
            dLdout = layer.backward(dLdout)
        return dLdout

    def __repr__(self) -> str:
        repr_str = "Network(\n"
        for layer in self.layers:
            repr_str += "    " + repr(layer) + "\n"
        repr_str += ')'
        return repr_str

### テスト
以下の２つが実行できれば実装は成功している。

In [ ]:
model = Network(input_dim=64, device=device)
input = torch.randn(2, 64, device=device)
print(model)
print('output=\n', model(input))
assert model.backward(torch.randn(2, 10, device=device)).shape == input.shape
print('backward works!')

In [ ]:
# Compute gradient for the first_linear.weight[845, 34] w.r.t. model(input).sum().

model.zero_grad()  # clear up the previously computed gradients
input = torch.randn(10, 64, device=device) * 10
model.backward(torch.ones_like(model(input)))  # for sum, dLdout is all ones.
backprop_grad = model.layers[0].weight.grad[845, 34]

print('Our manual backprop grad is ', backprop_grad.item())

eps = 1e-5
# -eps
model.layers[0].weight[845, 34] -= eps
output_0 = model(input).sum()
# +eps
model.layers[0].weight[845, 34] += 2 * eps
output_1 = model(input).sum()
# numerical
numerical_grad = (output_1 - output_0) / (2 * eps)

print('Numerical backprop grad is ', numerical_grad.item())
print('Any difference on the order of 1e-5 is fine.')

## 問３
AIがこのまま発展し続けた世界線において、我々はAIとどのように付き合っていけばよいかを考察せよ。


（解答欄）
